In [3]:
import numpy as np
import torch

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-17 14:35:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.30MB/s    in 0.3s    

2025-01-17 14:35:38 (3.30 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
from pathlib import Path

In [7]:
with Path("input.txt").open("r", encoding="utf-8") as f:
    text = f.read()

In [16]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
print(vocab_size)
print(vocab)

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [24]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [30]:
# take a string and output a list of ints
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: "".join([itos[i] for i in e])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [32]:
encode(text)

[18,
 47,
 56,
 57,
 58,
 1,
 15,
 47,
 58,
 47,
 64,
 43,
 52,
 10,
 0,
 14,
 43,
 44,
 53,
 56,
 43,
 1,
 61,
 43,
 1,
 54,
 56,
 53,
 41,
 43,
 43,
 42,
 1,
 39,
 52,
 63,
 1,
 44,
 59,
 56,
 58,
 46,
 43,
 56,
 6,
 1,
 46,
 43,
 39,
 56,
 1,
 51,
 43,
 1,
 57,
 54,
 43,
 39,
 49,
 8,
 0,
 0,
 13,
 50,
 50,
 10,
 0,
 31,
 54,
 43,
 39,
 49,
 6,
 1,
 57,
 54,
 43,
 39,
 49,
 8,
 0,
 0,
 18,
 47,
 56,
 57,
 58,
 1,
 15,
 47,
 58,
 47,
 64,
 43,
 52,
 10,
 0,
 37,
 53,
 59,
 1,
 39,
 56,
 43,
 1,
 39,
 50,
 50,
 1,
 56,
 43,
 57,
 53,
 50,
 60,
 43,
 42,
 1,
 56,
 39,
 58,
 46,
 43,
 56,
 1,
 58,
 53,
 1,
 42,
 47,
 43,
 1,
 58,
 46,
 39,
 52,
 1,
 58,
 53,
 1,
 44,
 39,
 51,
 47,
 57,
 46,
 12,
 0,
 0,
 13,
 50,
 50,
 10,
 0,
 30,
 43,
 57,
 53,
 50,
 60,
 43,
 42,
 8,
 1,
 56,
 43,
 57,
 53,
 50,
 60,
 43,
 42,
 8,
 0,
 0,
 18,
 47,
 56,
 57,
 58,
 1,
 15,
 47,
 58,
 47,
 64,
 43,
 52,
 10,
 0,
 18,
 47,
 56,
 57,
 58,
 6,
 1,
 63,
 53,
 59,
 1,
 49,
 52,
 53,
 61,
 1,
 15,
 39,
 47,

In [31]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [35]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [37]:
torch.manual_seed(1337)
batch_size = 4

In [38]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [54]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [58]:
# torch.randint(len(train_data) - block_size, (batch_size, ))
xb, yb = get_batch("train")
print(xb)
print(xb.shape)
print(yb)
print(yb.shape)

tensor([[52, 57,  8,  0, 21,  5, 50, 50],
        [47, 58, 46,  1, 63, 53, 59,  6],
        [ 1, 53, 40, 43, 63, 12,  0, 26],
        [50, 42, 57, 58,  6,  0, 32, 46]])
torch.Size([4, 8])
tensor([[57,  8,  0, 21,  5, 50, 50,  1],
        [58, 46,  1, 63, 53, 59,  6,  1],
        [53, 40, 43, 63, 12,  0, 26, 39],
        [42, 57, 58,  6,  0, 32, 46, 43]])
torch.Size([4, 8])


In [128]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        # Init at random a learnable embedding matrix
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # Get the prediction for the next token given the previous token(s)
            logits, _ = self(idx)
            # Select the predictions only
            logits = logits[:, -1, :] # (B, C)
            # Run the logits (value of embedding vector weights) through
            # softmax to get probability distribution
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution to get an index of a vector (a character)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenate the idx to the current idx to create the next
            # idx vector to pass into `self` (forward pass function)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [129]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(logits[:, -1])

torch.Size([32, 65])
tensor(4.8878, grad_fn=<NllLossBackward0>)
tensor([-0.9242, -1.4541,  1.1620,  0.2614,  0.2004, -0.5061, -0.4127, -0.4127,
        -1.0788, -0.4398,  0.4566,  0.9883,  0.6441,  0.3873,  0.6643,  0.9937,
         0.9883,  0.3873, -0.9960, -0.3203,  0.6441,  1.5383,  0.2614,  0.7179,
        -0.4127, -2.4499, -1.4541, -0.4398,  0.9937,  0.2614, -0.4957,  0.4566],
       grad_fn=<SelectBackward0>)


In [130]:
emb = nn.Embedding(vocab_size, vocab_size)
emb

Embedding(65, 65)

In [131]:
# emb will lookup and return a tensor of shape (m, vocab_size) or ((B, T), vocab_size)
# of the vectors corresponding to the indices of the tensor you pass in
# val = emb(torch.tensor((5, 39, 2, 58, 29)))
# print(val.shape)

In [132]:
in_ = torch.zeros((1, 1), dtype=torch.long)
print(in_)

tensor([[0]])


In [133]:
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[-0.9218, -0.0479,  0.3535,  0.7331, -0.2906,  0.0619,  1.0745, -0.0425,
         -0.1016,  0.4784, -0.3332, -0.6770,  0.0819,  1.4632, -0.1850,  0.1062,
         -1.1108,  1.7659,  1.3204, -0.3736, -1.1037, -0.5767,  1.3159, -0.1677,
         -0.3398, -1.7412,  0.2524,  0.4322, -0.2262,  1.6449,  0.3119,  0.1886,
         -0.7560, -0.3299, -0.6503, -0.2238,  0.0681,  1.1422, -1.1548, -1.8727,
         -0.8486, -0.4800,  0.0787, -0.7758, -0.3702,  1.2381, -2.6668, -0.1514,
         -0.1855, -0.4004,  1.9810,  0.0878,  1.1793,  0.6098,  1.2850,  0.6165,
          1.9138,  0.5396,  0.1724, -2.2152, -1.0297,  0.0716, -0.1826,  0.7390,
          0.2614]], grad_fn=<SliceBackward0>)
tensor([[ 1.3960, -1.1297, -0.3686,  0.3818,  0.1234, -0.4650, -0.3006, -1.1808,
         -0.7408,  0.0736,  1.2068,  1.1722,  0.8508, -0.8872,  0.5344, -1.1583,
          0.6022,  0.8180, -0.0017,  0.6875,  0.4724, -1.1652,  0.0093,  0.1120,
         -0.4605, -0.4103, -1.3358,  0.3569, -0.2399, -0.1669, 